In [24]:
import pandas as pd
import re
from datasets import Dataset

# Load data

In [4]:
data_path = '../data/naruto.csv'
naruto_transcript_df = pd.read_csv(data_path)
naruto_transcript_df.head()

,name,line
0,Naruto,(Laughing) Give it up. (Shows the stone faces...
1,Hiruzen,(Turns away from his writing) I hope you’re n...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [7]:
# Remove actions from transcript
def remove_paranthesis(text):
    result = re.sub(r'\(.*?\)', '', text)
    return result

In [9]:
naruto_transcript_df['line'] = naruto_transcript_df['line'].apply(remove_paranthesis)
naruto_transcript_df.head()

,name,line
0,Naruto,"Give it up. You’re just bent, because you d..."
1,Hiruzen,I hope you’re not bothering me with some tri...
2,Ninja,Naseer Sabah
3,Ninja,is the best person on earth
4,Naruto,muah


In [12]:
naruto_transcript_df['no_of_words'] = naruto_transcript_df['line'].str.strip().str.split(' ')
naruto_transcript_df['no_of_words'] = naruto_transcript_df['no_of_words'].apply(lambda x:len(x))
naruto_transcript_df.head()

,name,line,no_of_words
0,Naruto,"Give it up. You’re just bent, because you d...",26
1,Hiruzen,I hope you’re not bothering me with some tri...,16
2,Ninja,Naseer Sabah,2
3,Ninja,is the best person on earth,6
4,Naruto,muah,1


In [13]:
naruto_transcript_df['naruto_response_flag'] = 0
naruto_transcript_df.loc[(naruto_transcript_df['name']=='Naruto')&(naruto_transcript_df['no_of_words']>5),'naruto_response_flag']=1
naruto_transcript_df

,name,line,no_of_words,naruto_response_flag
0,Naruto,"Give it up. You’re just bent, because you d...",26,1
1,Hiruzen,I hope you’re not bothering me with some tri...,16,0
2,Ninja,Naseer Sabah,2,0
3,Ninja,is the best person on earth,6,0
4,Naruto,muah,1,0
...,...,...,...,...
158,Iruka,Congratulations. You graduate. Naruto’s stand...,27,0
159,Iruka,Huh?,1,0
160,Naruto,Iruka Sensei!,2,0
161,Iruka,Ah! That hurts!,3,0


In [15]:
indexes_to_take = list(naruto_transcript_df[(naruto_transcript_df['naruto_response_flag']==1)&(naruto_transcript_df.index>0)].index)
indexes_to_take[:5]

[6, 28, 30, 32, 36]

In [21]:
system_promt = """You are Naruto from the anime "Naruto". Your responses should reflect his personality and speech patterns.\n"""

promts = []
for ind in indexes_to_take:
    promt = system_promt
    
    promt += naruto_transcript_df.iloc[ind-1]['line']
    promt += '\n'
    promt += naruto_transcript_df.iloc[ind]['line']
    promts.append(promt)

In [22]:
print(promts[3])

You are Naruto from the anime "Naruto". Your responses should reflect his personality and speech patterns.
 Well, I was just thinking. Maybe after you clean this all up I can take you out for some ramen. The good stuff. What do you think? 
  Now that’s some serious motivation! I’ll have this clean in no time!


In [23]:
df = pd.DataFrame({'promt':promts})
df.head()

,promt
0,"You are Naruto from the anime ""Naruto"". Your r..."
1,"You are Naruto from the anime ""Naruto"". Your r..."
2,"You are Naruto from the anime ""Naruto"". Your r..."
3,"You are Naruto from the anime ""Naruto"". Your r..."
4,"You are Naruto from the anime ""Naruto"". Your r..."


In [25]:
dataset = Dataset.from_pandas(df)